In [1]:
suppressMessages(library(repr))
options(repr.plot.width=25,
        repr.plot.height=10,
        #repr.plot.pointsize=50,
        repr.plot.family='serif'
       )

In [2]:
setwd("~/Repositories/AirQualityCOVID/")

In [3]:
source("src/Analisis/Variacion/functions.R")
source("src/general.R")

# Data variables

In [4]:
municipio = "Santander"
site = "" # c("es1580a", "es0118a")

data.by.file = TRUE

pollutant = c("no")#, "no2", "o3", "pm10", "pm2.5")
start_dt = 2013

# Lockdown dates

In [5]:
periods <- list(
    # lockdown
    lockdown = as.numeric(ymd_hms("2020-03-14 00:00:00")),
    #end.lockdown = as.numeric(ymd_hms("2020-04-28 00:00:00")),

    # poslockdown
    fases = as.numeric(ymd_hms("2020-05-01 00:00:00")),
    normalidad = as.numeric(ymd_hms("2020-06-21 00:00:00")),

    # New lockdown
    new.lockdown = as.numeric(ymd_hms("2020-10-25 00:00:00"))
)

---

In [6]:
if (site == "" && municipio != "") {
    site <- get.site.municipio(municipio,
                               fileName="data/Curation/sitesAQ.csv")
}

In [7]:
data.AQ <- get.AQdata(site, pollutant, start_dt, end_dt=2020, data.by.file)
head(data.AQ)

date,date_end,site,variable,process,summary,validity,unit,value
<dttm>,<dttm>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>
2013-01-01 00:00:00,2013-01-01 01:00:00,es1580a,no,118052,1,1,ug.m-3,7
2013-01-01 01:00:00,2013-01-01 02:00:00,es1580a,no,118052,1,1,ug.m-3,2
2013-01-01 02:00:00,2013-01-01 03:00:00,es1580a,no,118052,1,1,ug.m-3,1
2013-01-01 03:00:00,2013-01-01 04:00:00,es1580a,no,118052,1,1,ug.m-3,1
2013-01-01 04:00:00,2013-01-01 05:00:00,es1580a,no,118052,1,1,ug.m-3,0
2013-01-01 05:00:00,2013-01-01 06:00:00,es1580a,no,118052,1,1,ug.m-3,1


# Calculo Propio de Media diaria

In [8]:
data.AQ.dy <- data.AQ[complete.cases(data.AQ),] #%>%
                #group.dy(unit="day", FUN=mean)

In [9]:
head(data.AQ.dy)

date,date_end,site,variable,process,summary,validity,unit,value
<dttm>,<dttm>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>
2013-01-01 00:00:00,2013-01-01 01:00:00,es1580a,no,118052,1,1,ug.m-3,7
2013-01-01 01:00:00,2013-01-01 02:00:00,es1580a,no,118052,1,1,ug.m-3,2
2013-01-01 02:00:00,2013-01-01 03:00:00,es1580a,no,118052,1,1,ug.m-3,1
2013-01-01 03:00:00,2013-01-01 04:00:00,es1580a,no,118052,1,1,ug.m-3,1
2013-01-01 04:00:00,2013-01-01 05:00:00,es1580a,no,118052,1,1,ug.m-3,0
2013-01-01 05:00:00,2013-01-01 06:00:00,es1580a,no,118052,1,1,ug.m-3,1


# Comparar Datos por Año

In [10]:
data.all <- compare.years(data.AQ.dy, last.yr=2020)

In [11]:
data.all <- data.all[complete.cases(data.all),]

# Variaciones diarias

In [31]:
# Calcular variaciones
var.med <- get.difference(data.all, "2020",c(paste(start_dt, "-2019", sep="")))
var.med[,4:ncol(var.med)] <- var.med[,4:ncol(var.med)]*100

var.med <- var.med[complete.cases(var.med), ]

In [32]:
length(var.med$date)
length(var.med$"2013-2019.vs.2020")

[1] 7993

[1] 7993

# Obtain Breakpoints from Variations

In [33]:
var <- data.frame(value=var.med$"2013-2019.vs.2020",
                  date=var.med$date
                 )
var <- var[-which(var$value == Inf), ]
var <- var[-which(var$value == -Inf), ]

In [34]:
var$date <- floor_date(var$date, unit="day")

In [35]:
head(var)

value,date
<dbl>,<dttm>


In [36]:
var <- aggregate(list(value=var$value), by=list(date=var$date), mean, na.rm=TRUE)

ERROR: Error in aggregate.data.frame(as.data.frame(x), ...): no rows to aggregate


In [37]:
var <- filter.by.std(var, c("value"), 1)

In [26]:
max(var$y, na.rm=TRUE)

Warning message in max(var$y, na.rm = TRUE):
"no non-missing arguments to max; returning -Inf"


[1] -Inf

## Using `segmented` package

In [ ]:
suppressMessages(library(segmented))

my.seg <- segmented(lm(y ~ 1 + x, data=var), 
                      seg.Z = ~ x,
                      psi = list(x = periods)
                     )

# Tendencia en los intervalos obtenidos

In [ ]:
suppressMessages(library(mblm))
a <- mblm(y ~ x + 1,
          var[var$x < my.seg$psi[1],],
          repeated=T)

# Representar Datos

In [ ]:
plot <- ggplot(data=var, aes(x=as_datetime(x)))+ 
    scale_color_manual(name = "Years", 
                       values = c("2010-2019" = "red",
                                  "2019" = "blue"))

#for (k in 1:(nrow(my.seg$psi))) {
    plot <- plot + geom_line(aes(y=y, color="2010-2019"), size=0.2)
    
#    plot <- plot + geom_vline(xintercept = my.seg$psi[k,"Est."], 
#                              linetype=5, color = "red", size=1.5)
#    plot <- plot + geom_vline(xintercept = my.seg$psi[k,"Est."] - my.seg$psi[k,"St.Err"], 
#                              linetype=5, color = "orange", size=1)
#    plot <- plot + geom_vline(xintercept = my.seg$psi[k,"Est."] + my.seg$psi[k,"St.Err"], 
#                              linetype=5, color = "orange", size=1)
#}

for (k in 1:(length(periods))) {
    plot <- plot + geom_vline(xintercept = as.numeric(periods[k]), 
                              linetype=4, color = "black", size=1)
}


plot